## In Deep Learning

* Many layers: compositionality
* Convolutions: locality + stationarity of images
* Pooling: Invariance of object class to translations

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
import matplotlib.pyplot as plt
import numpy

def get_n_params(model):
    np = 0
    for p in list(model.parameters()):
        np += p.nelement()
    return np

In [3]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

C:\Users\wilso\Anaconda3\envs\ia-tools\lib\site-packages\torch\cuda\__init__.py:52: UserWarning: CUDA initialization: The NVIDIA driver on your system is too old (found version 10010). Please update your GPU driver by downloading and installing a new version from the URL: http://www.nvidia.com/Download/index.aspx Alternatively, go to: https://pytorch.org to install a PyTorch version that has been compiled with your version of the CUDA driver. (Triggered internally at  ..\c10\cuda\CUDAFunctions.cpp:100.)
  return torch._C._cuda_getDeviceCount() > 0


## Load MNIST

In [4]:
input_size = 28*28
output_size = 10

train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=64, shuffle=True)


test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
        transforms.ToTensor(),
        transforms.Normalize((0.1307,), (0.3081,))
    ])), batch_size=1000, shuffle=True)

Extracting ../data\MNIST\raw\train-images-idx3-ubyte.gz to ../data\MNIST\raw


Extracting ../data\MNIST\raw\train-labels-idx1-ubyte.gz to ../data\MNIST\raw


Extracting ../data\MNIST\raw\t10k-images-idx3-ubyte.gz to ../data\MNIST\raw




Extracting ../data\MNIST\raw\t10k-labels-idx1-ubyte.gz to ../data\MNIST\raw
Processing...


C:\Users\wilso\Anaconda3\envs\ia-tools\lib\site-packages\torchvision\datasets\mnist.py:480: UserWarning: The given NumPy array is not writeable, and PyTorch does not support non-writeable tensors. This means you can write to the underlying (supposedly non-writeable) NumPy array using the tensor. You may want to copy the array to protect its data or make it writeable before converting it to a tensor. This type of warning will be suppressed for the rest of this program. (Triggered internally at  ..\torch\csrc\utils\tensor_numpy.cpp:141.)
  return torch.from_numpy(parsed.astype(m[2], copy=False)).view(*s)


Done!


In [5]:
class CNN(nn.Module):
    def __init__(self, input_size, n_feature, output_size):
        super(CNN, self).__init__()
        
        self.n_feature = n_feature
        self.conv1 = nn.Conv2d(in_channels=1, out_channels=n_feature, kernel_size=5)
        self.conv2 = nn.Conv2d(n_feature, n_feature, kernel_size=5)
        self.fc1 = nn.Linear(n_feature*4*4, 50)
        self.fc2 = nn.Linear(50, 10)
        
        
    def forward(self, x, verbose=False):
        
        x = self.conv1(x)
        x = F.relu(x)        
        x = F.max_pool2d(x, kernel_size=2)
        
        x = self.conv2(x)
        x = F.relu(x)
        x = F.max_pool2d(x, kernel_size=2)
        
        x = x.view(-1, self.n_feature*4*4)
        
        x = self.fc1(x)
        x = F.relu(x)
        
        x = self.fc2(x)
        x = F.log_softmax(x, dim=1)
        
        return x

In [6]:
accuracy_list = []

def train(epoch, model, perm=torch.arange(0, 784).long()):
    
    model.train()
    
    for batch_idx, (data, target) in enumerate(train_loader):
        # send to device
        data, target = data.to(device), target.to(device)
        
        optimizer.zero_grad()
        output = model(data)
            
        loss = F.nll_loss(output, target)
        loss.backward()
        
        optimizer.step()
        
        if batch_idx % 100 == 0:
            print("Train epoch: {} [{}/{} ({:.0f}%)\t Loss: {:.6f}]".format(epoch, batch_idx*len(data), 
                                                                            len(train_loader.dataset),
                                                                           100.*batch_idx/len(train_loader), loss.item()))

def test(model):
    model.eval()
    
    test_loss = 0
    correct = 0
    
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)
        
        output = model(data)
        test_loss += F.nll_loss(output, target, reduction='sum').item()
        pred = output.data.max(1, keepdim=True)[1]
        correct += pred.eq(target.data.view_as(pred)).cpu().sum().item()
        
    test_loss /= len(test_loader.dataset)
    accuracy = 100. * correct / len(test_loader.dataset)
    accuracy_list.append(accuracy)
    
    print("\nTest set: Average loss: {:.4f}, Accuracy: {}/{} ({:0f}%)\n".format(test_loss, correct, len(test_loader.dataset), accuracy))
    
        


In [7]:
n_features = 6

model_cnn = CNN(input_size, n_features, output_size)
model_cnn.to(device)

optimizer = optim.SGD(model_cnn.parameters(), lr=0.01, momentum=0.5)
print('Number of parameters: {}'.format(get_n_params(model_cnn)))

for epoch in range(0, 1):
    train(epoch, model_cnn)
    test(model_cnn)

Number of parameters: 6422
Train epoch: 0 [0/60000 (0%)	 Loss: 2.333160]
Train epoch: 0 [6400/60000 (11%)	 Loss: 2.059641]
Train epoch: 0 [12800/60000 (21%)	 Loss: 0.542242]
Train epoch: 0 [19200/60000 (32%)	 Loss: 0.581156]
Train epoch: 0 [25600/60000 (43%)	 Loss: 0.428636]
Train epoch: 0 [32000/60000 (53%)	 Loss: 0.294578]
Train epoch: 0 [38400/60000 (64%)	 Loss: 0.240659]
Train epoch: 0 [44800/60000 (75%)	 Loss: 0.076033]
Train epoch: 0 [51200/60000 (85%)	 Loss: 0.192906]
Train epoch: 0 [57600/60000 (96%)	 Loss: 0.284516]

Test set: Average loss: 0.1833, Accuracy: 9419/10000 (94.190000%)

